In [1]:
!pip install ipywidgets


In [3]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

# Replace 'your_path_here' with the path to your file
file_path = 'Carbonation database.xlsx'  #'/content/drive/MyDrive/DrBehnoodResearch/Carbonation database.xlsx'
df = pd.read_excel(file_path, sheet_name='Carbonation')

# Display the dataframe
# Drop the columns 'Ref' and 'Sample ID'
df = df.drop(columns=['Ref', 'Sample ID'])
# Calculate mean of each column and replace NaN with mean
df['Temp\n(C)'] = df['Temp\n(C)'].fillna(df['Temp\n(C)'].mean())
df['RH\n(%)'] = df['RH\n(%)'].fillna(df['RH\n(%)'].mean())
# df = df.dropna(subset=['Temp\n(C)', 'RH\n(%)'])
df

,C\n(kg/m3),FlA (kg/m3),Water\n(kg/m3),Gravel (kg/m3),RA \n(kg/m3),Equivalent WA of CA (%),FA\n(kg/m3),SP\n(kg/m3),Cube CS_28\n(MPa),Carbon concentration (%),Exposure time (days),Temp\n(C),RH\n(%),Carbonation depth (mm)
0,361.1,0.0,182.0,981.7080,0.00000,1.400000,357.655,0.0,53.9,5.0,7.0,21.777695,63.512236,2.7000
1,361.1,0.0,182.0,883.5372,78.83070,1.989776,357.655,0.0,54.1,5.0,7.0,21.777695,63.512236,2.7000
2,361.1,0.0,182.0,736.2810,197.07675,2.920267,357.655,0.0,48.9,5.0,7.0,21.777695,63.512236,3.9000
3,361.1,0.0,182.0,490.8540,394.15350,4.606645,357.655,0.0,46.2,5.0,7.0,21.777695,63.512236,3.5000
4,361.1,0.0,182.0,0.0000,788.30700,8.600000,357.655,0.0,35.3,5.0,7.0,21.777695,63.512236,5.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,333.3,0.0,200.0,566.8000,519.00000,3.011074,694.200,0.0,36.4,20.0,70.0,20.000000,60.000000,29.1836
815,410.0,0.0,205.0,0.0000,1006.60000,5.120000,643.500,0.0,31.1,20.0,70.0,20.000000,60.000000,28.9671
816,274.6,0.0,195.0,1131.1000,0.00000,1.080000,754.100,0.0,37.5,20.0,84.0,20.000000,60.000000,19.8106
817,333.3,0.0,200.0,566.8000,519.00000,3.011074,694.200,0.0,36.4,20.0,84.0,20.000000,60.000000,31.0149


In [5]:
df.columns

Index(['C\n(kg/m3)', 'FlA (kg/m3)', 'Water\n(kg/m3)', 'Gravel (kg/m3)',
       'RA \n(kg/m3)', 'Equivalent  WA of CA (%)', 'FA\n(kg/m3)',
       'SP\n(kg/m3)', 'Cube CS_28\n(MPa)', 'Carbon concentration (%)',
       'Exposure time (days)', 'Temp\n(C)', 'RH\n(%)',
       'Carbonation depth (mm)'],
      dtype='object')

In [7]:
!pip install xgboost


In [9]:
import numpy as np
from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Layout, Label
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from scipy.spatial.distance import cdist

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()
y=y.reshape(-1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)

In [13]:
def jackknife(X, Y, Xtest, alpha):
    n = X.shape[0]
    ntest = Xtest.shape[0]
    R = np.zeros(n)
    for i in np.arange(n):
        X_ = np.delete(X, i, 0)
        Y_ = np.delete(Y, i)
        pipeline.fit(X_, Y_)
        R[i] = np.abs(Y[i] - pipeline.predict(X[i].reshape(1, -1)).item())
    PIs = np.zeros((ntest, 2))
    y_pred = pipeline.predict(Xtest)
    for itest in np.arange(ntest):
        q_lo = np.sort(y_pred[itest] - R)[::-1][(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        q_hi = np.sort(y_pred[itest] + R)[(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        PIs[itest] = np.array([q_lo, q_hi])
    return PIs

def jackknife_plus(X, Y, Xtest, alpha):
    n = X.shape[0]
    ntest = Xtest.shape[0]
    R = np.zeros(n)
    y_pred = np.zeros((n, ntest))
    for i in np.arange(n):
        X_ = np.delete(X, i, 0)
        Y_ = np.delete(Y, i)
        pipeline.fit(X_, Y_)
        R[i] = np.abs(Y[i] - pipeline.predict(X[i].reshape(1, -1)).item())
        y_pred[i] = pipeline.predict(Xtest)
    PIs = np.zeros((ntest, 2))
    for itest in np.arange(ntest):
        q_lo = np.sort(y_pred[:, itest] - R)[::-1][(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        q_hi = np.sort(y_pred[:, itest] + R)[(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        PIs[itest] = np.array([q_lo, q_hi])
    return PIs

def CV(X, Y, Xtest, alpha, K_CV=10):
    n = X.shape[0]
    m = int(n / K_CV)
    ntest = Xtest.shape[0]
    R = np.zeros(n)
    for k in np.arange(K_CV):
        fold = np.arange(k * m, (k + 1) * m)
        X_ = np.delete(X, fold, 0)
        Y_ = np.delete(Y, fold)
        pipeline.fit(X_, Y_)
        R[fold] = np.abs(Y[fold] - pipeline.predict(X[fold]).reshape(-1))
    PIs = np.zeros((ntest, 2))
    y_pred = pipeline.predict(Xtest)
    for itest in np.arange(ntest):
        q_lo = np.sort(y_pred[itest] - R)[::-1][(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        q_hi = np.sort(y_pred[itest] + R)[(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        PIs[itest] = np.array([q_lo, q_hi])
    return PIs

def CV_plus(X, Y, Xtest, alpha, K_CV=10):
    n = X.shape[0]
    m = int(n / K_CV)
    ntest = Xtest.shape[0]
    R = np.zeros(n)
    y_pred = np.zeros((n, ntest))
    for k in np.arange(K_CV):
        fold = np.arange(k * m, (k + 1) * m)
        X_ = np.delete(X, fold, 0)
        Y_ = np.delete(Y, fold)
        pipeline.fit(X_, Y_)
        R[fold] = np.abs(Y[fold] - pipeline.predict(X[fold]).reshape(-1))
        y_pred[fold] = pipeline.predict(Xtest)
    PIs = np.zeros((ntest, 2))
    for itest in np.arange(ntest):
        q_lo = np.sort(y_pred[:, itest] - R)[::-1][(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        q_hi = np.sort(y_pred[:, itest] + R)[(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        PIs[itest] = np.array([q_lo, q_hi])
    return PIs

def weighted_jackknife_plus(X, Y, Xtest, alpha):
    n = X.shape[0]
    ntest = Xtest.shape[0]
    R = np.zeros(n)
    y_pred = np.zeros((n, ntest))
    d = np.zeros((n, ntest))
    for i in np.arange(n):
        X_ = np.delete(X, i, 0)
        Y_ = np.delete(Y, i)
        pipeline.fit(X_, Y_)
        R[i] = np.abs(Y[i] - pipeline.predict(X[i].reshape(1, -1)).item())
        y_pred[i] = pipeline.predict(Xtest)
        d[i] = 1 / (cdist(X[i].reshape(1, -1), Xtest, 'euclidean'))
    PIs = np.zeros((ntest, 2))
    for itest in np.arange(ntest):
        w = d[:, itest] / d[:, itest].sum()
        q_lo = np.sort(y_pred[:, itest] - (w * R * n))[::-1][(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        q_hi = np.sort(y_pred[:, itest] + (w * R * n))[(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        PIs[itest] = np.array([q_lo, q_hi])
    return PIs

def weighted_CV_plus(X, Y, Xtest, alpha, K_CV=10):
    n = X.shape[0]
    m = int(n / K_CV)
    ntest = Xtest.shape[0]
    R = np.zeros(n)
    y_pred = np.zeros((n, ntest))
    d = np.zeros((n, ntest))
    for k in np.arange(K_CV):
        fold = np.arange(k * m, (k + 1) * m)
        X_ = np.delete(X, fold, 0)
        Y_ = np.delete(Y, fold)
        pipeline.fit(X_, Y_)
        R[fold] = np.abs(Y[fold] - pipeline.predict(X[fold]).reshape(-1))
        y_pred[fold] = pipeline.predict(Xtest)
        d[fold] = 1 / (cdist(X[fold], Xtest, 'euclidean'))
    PIs = np.zeros((ntest, 2))
    for itest in np.arange(ntest):
        w = d[:, itest] / d[:, itest].sum()
        q_lo = np.sort(y_pred[:, itest] - (w * R * n))[::-1][(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        q_hi = np.sort(y_pred[:, itest] + (w * R * n))[(np.ceil((1 - alpha) * (n + 1))).astype(int)]
        PIs[itest] = np.array([q_lo, q_hi])
    return PIs


In [12]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Title for GUI
gui_title = ttk.Label(main_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Labels and input sliders
inputs_frame = ttk.LabelFrame(main_frame, text="Input Features", padding="10")
inputs_frame.grid(row=1, column=0, columnspan=4, sticky=(tk.W, tk.E))

entries = {}
for idx, label in enumerate(input_labels):
    ttk.Label(inputs_frame, text=label).grid(row=idx, column=0, sticky=tk.W)
    default, min_val, max_val = stat_summary[label]
    slider = ttk.Scale(inputs_frame, from_=min_val, to=max_val, orient=tk.HORIZONTAL)
    slider.set(default)
    slider.grid(row=idx, column=1, sticky=(tk.W, tk.E))
    entries[label] = slider

# Confidence level
confidence_label = ttk.Label(main_frame, text="Confidence Level")
confidence_label.grid(row=2, column=0, pady=(20, 5), sticky=tk.W)
confidence_level = ttk.Scale(main_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL)
confidence_level.set(0.95)
confidence_level.grid(row=2, column=1, pady=(20, 5), sticky=(tk.W, tk.E))

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(main_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=3, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))

methods = {
    'jackknife': tk.StringVar(value='J'),
    'jackknife_plus': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV_plus': tk.StringVar(value='CV+'),
    'weighted_jackknife_plus': tk.StringVar(value='WJ+'),
    'weighted_CV_plus': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('jackknife')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(main_frame, text="Output", padding="10")
output_frame.grid(row=4, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=5, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[slider.get() for slider in entries.values()]])
    alpha = 1 - confidence_level.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, pd.DataFrame(formatted_PIs, columns=['Lower Bound', 'Upper Bound']).to_string())
    output_text.insert(tk.END, f"\nMidpoint: {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(main_frame, text="Compute", command=display_results)
compute_button.grid(row=5, column=0, pady=(20, 5))

reset_button = ttk.Button(main_frame, text="Reset", command=lambda: [slider.set(stat_summary[label][0]) for label, slider in entries.items()])
reset_button.grid(row=5, column=1, pady=(20, 5))

exit_button = ttk.Button(main_frame, text="Exit", command=root.destroy)
exit_button.grid(row=5, column=2, pady=(20, 5))

root.mainloop()


In [ ]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Title for GUI
gui_title = ttk.Label(main_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Labels and input sliders
inputs_frame = ttk.LabelFrame(main_frame, text="Input Features", padding="10")
inputs_frame.grid(row=1, column=0, columnspan=4, sticky=(tk.W, tk.E))

entries = {}
entry_vars = {}
row = 0
col = 0
for idx, label in enumerate(input_labels):
    if idx == 7:
        row = 0
        col = 2
    ttk.Label(inputs_frame, text=label).grid(row=row, column=col, sticky=tk.W)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    slider = ttk.Scale(inputs_frame, from_=min_val, to=max_val, orient=tk.HORIZONTAL, variable=slider_var)
    slider.grid(row=row, column=col+1, sticky=(tk.W, tk.E))
    entries[label] = slider
    ttk.Label(inputs_frame, textvariable=slider_var).grid(row=row, column=col+2, sticky=tk.W)
    row += 1

# Confidence level
confidence_label = ttk.Label(main_frame, text="Confidence Level")
confidence_label.grid(row=2, column=0, pady=(20, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(main_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=2, column=1, pady=(20, 5), sticky=(tk.W, tk.E))
ttk.Label(main_frame, textvariable=confidence_var).grid(row=2, column=2, pady=(20, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(main_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=3, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))

methods = {
    'jackknife': tk.StringVar(value='J'),
    'jackknife_plus': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV_plus': tk.StringVar(value='CV+'),
    'weighted_jackknife_plus': tk.StringVar(value='WJ+'),
    'weighted_CV_plus': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('jackknife')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(main_frame, text="Output", padding="10")
output_frame.grid(row=4, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=5, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[slider.get() for slider in entries.values()]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, pd.DataFrame(formatted_PIs, columns=['Lower Bound', 'Upper Bound']).to_string())
    output_text.insert(tk.END, f"\nMidpoint: {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(main_frame, text="Compute", command=display_results)
compute_button.grid(row=5, column=0, pady=(20, 5))

reset_button = ttk.Button(main_frame, text="Reset", command=lambda: [slider.set(stat_summary[label][0]) for label, slider in entries.items()])
reset_button.grid(row=5, column=1, pady=(20, 5))

exit_button = ttk.Button(main_frame, text="Exit", command=root.destroy)
exit_button.grid(row=5, column=2, pady=(20, 5))

root.mainloop()


In [9]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Title for GUI
gui_title = ttk.Label(main_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Labels and input sliders
inputs_frame = ttk.LabelFrame(main_frame, text="Input Features", padding="10")
inputs_frame.grid(row=1, column=0, columnspan=4, sticky=(tk.W, tk.E))

entries = {}
entry_vars = {}
row = 0
col = 0
for idx, label in enumerate(input_labels):
    if idx == 7:
        row = 0
        col = 2
    ttk.Label(inputs_frame, text=label).grid(row=row, column=col, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    slider = ttk.Scale(inputs_frame, from_=min_val, to=max_val, orient=tk.HORIZONTAL, variable=slider_var)
    slider.grid(row=row, column=col+1, sticky=(tk.W, tk.E), padx=5, pady=5)
    entries[label] = slider
    ttk.Label(inputs_frame, textvariable=slider_var).grid(row=row, column=col+2, sticky=tk.W, padx=5, pady=5)
    row += 1

# Confidence level
confidence_label = ttk.Label(main_frame, text="Confidence Level")
confidence_label.grid(row=2, column=0, pady=(20, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(main_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=2, column=1, pady=(20, 5), sticky=(tk.W, tk.E))
ttk.Label(main_frame, textvariable=confidence_var).grid(row=2, column=2, pady=(20, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(main_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=3, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))

methods = {
    'jackknife': tk.StringVar(value='J'),
    'jackknife_plus': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV_plus': tk.StringVar(value='CV+'),
    'weighted_jackknife_plus': tk.StringVar(value='WJ+'),
    'weighted_CV_plus': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('jackknife')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(main_frame, text="Output", padding="10")
output_frame.grid(row=4, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[slider.get() for slider in entries.values()]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], Midpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(main_frame, text="Compute", command=display_results)
compute_button.grid(row=5, column=0, pady=(20, 5))

reset_button = ttk.Button(main_frame, text="Reset", command=lambda: [slider.set(stat_summary[label][0]) for label, slider in entries.items()])
reset_button.grid(row=5, column=1, pady=(20, 5))

exit_button = ttk.Button(main_frame, text="Exit", command=root.destroy)
exit_button.grid(row=5, column=2, pady=(20, 5))

root.mainloop()


In [11]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Title for GUI
gui_title = ttk.Label(main_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Labels and input sliders
inputs_frame = ttk.LabelFrame(main_frame, text="Input Features", padding="10")
inputs_frame.grid(row=1, column=0, columnspan=4, sticky=(tk.W, tk.E))

entries = {}
entry_vars = {}
row = 0
col = 0
for idx, label in enumerate(input_labels):
    if idx == 7:
        row = 0
        col = 2
    ttk.Label(inputs_frame, text=label).grid(row=row, column=col, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    slider = ttk.Scale(inputs_frame, from_=min_val, to=max_val, orient=tk.HORIZONTAL, variable=slider_var)
    slider.grid(row=row, column=col+1, sticky=(tk.W, tk.E), padx=5, pady=5)
    entry = ttk.Entry(inputs_frame, textvariable=slider_var, width=8)
    entry.grid(row=row, column=col+2, sticky=tk.W, padx=5, pady=5)
    entries[label] = slider
    row += 1

# Confidence level
confidence_label = ttk.Label(main_frame, text="Confidence Level")
confidence_label.grid(row=2, column=0, pady=(20, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(main_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=2, column=1, pady=(20, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(main_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=2, column=2, pady=(20, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(main_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=3, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))

methods = {
    'jackknife': tk.StringVar(value='J'),
    'jackknife_plus': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV_plus': tk.StringVar(value='CV+'),
    'weighted_jackknife_plus': tk.StringVar(value='WJ+'),
    'weighted_CV_plus': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('jackknife')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(main_frame, text="Output", padding="10")
output_frame.grid(row=4, column=0, columnspan=4, pady=(20, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[slider.get() for slider in entries.values()]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], \nMidpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(main_frame, text="Compute", command=display_results)
compute_button.grid(row=5, column=0, pady=(20, 5))

reset_button = ttk.Button(main_frame, text="Reset", command=lambda: [slider.set(stat_summary[label][0]) for label, slider in entries.items()])
reset_button.grid(row=5, column=1, pady=(20, 5))

exit_button = ttk.Button(main_frame, text="Exit", command=root.destroy)
exit_button.grid(row=5, column=2, pady=(20, 5))

root.mainloop()


## ####This one

In [45]:
####This one
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
row = 2
for idx, label in enumerate(input_labels):
    ttk.Label(left_frame, text=label).grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
    ttk.Label(left_frame, text=f"{min_val}").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
    ttk.Label(left_frame, text=f"{max_val}").grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': tk.StringVar(value='J'),
    'J+': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV_+': tk.StringVar(value='CV+'),
    'wWJ+': tk.StringVar(value='WJ+'),
    'WCV+': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('jackknife')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Output", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], \nMidpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))

exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()


## The best

In [15]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Units for each input feature
units = {
    'C': '(kg/m\u00b3)', 'FLA': '(kg/m\u00b3)', 'W': '(kg/m\u00b3)', 'CA': '(kg/m\u00b3)', 'RA': '(kg/m\u00b3)', 'RWA': '(%)',
    'FA': '(kg/m\u00b3)', 'SP': '(kg/m\u00b3)', 'CS': '(MPa)', 'CC': '(%)', 'ET': '(days)', 'T': '(°C)', 'RH': '(%)'
}


# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

# Create a frame to hold the title
title_frame = ttk.Frame(root, padding="10")
title_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Create the title label with bold and larger font
title_label = tk.Label(title_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 20, "bold"))
title_label.pack()



main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
row = 2
for idx, label in enumerate(input_labels):
    unit = units[label]
    ttk.Label(left_frame, text=f"{label} {unit}").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
    ttk.Label(left_frame, text=f"{min_val}").grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
    ttk.Label(left_frame, text=f"{max_val}").grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
# gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
# gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': tk.StringVar(value='J'),
    'J+': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV+': tk.StringVar(value='CV+'),
    'WJ+': tk.StringVar(value='WJ+'),
    'WCV+': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('J')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Carbonation Depth (mm)", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'J':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'J+':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'WJ+':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV+':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'WCV+':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}] \nMidpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))

exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()


In [43]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Units for each input feature
units = {
    'C': '(kg/m^3)', 'FLA': '(kg/m^3)', 'W': '(kg/m^3)', 'CA': '(kg/m^3)', 'RA': '(kg/m^3)', 'RWA': '(%)',
    'FA': '(kg/m^3)', 'SP': '(kg/m^3)', 'CS': '(MPa)', 'CC': '(%)', 'ET': '(days)', 'T': '(C)', 'RH': '(%)'
}

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=5, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=4, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
row = 2
for idx, label in enumerate(input_labels):
    unit = units[label]
    ttk.Label(left_frame, text=f"{label} {unit}").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
    min_var = tk.StringVar(value=f"{min_val}")
    min_entry = ttk.Entry(left_frame, textvariable=min_var, width=10, state='readonly')
    min_entry.grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    max_var = tk.StringVar(value=f"{max_val}")
    max_entry = ttk.Entry(left_frame, textvariable=max_var, width=10, state='readonly')
    max_entry.grid(row=row, column=4, sticky=tk.W, padx=5, pady=5)
    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': tk.StringVar(value='J'),
    'J+': tk.StringVar(value='J+'),
    'CV': tk.StringVar(value='CV'),
    'CV_+': tk.StringVar(value='CV+'),
    'wWJ+': tk.StringVar(value='WJ+'),
    'WCV+': tk.StringVar(value='WCV+')
}

selected_method = tk.StringVar()
selected_method.set('J')

for idx, (method, var) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=method, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Output (mm)", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'J':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'J+':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'WJ+':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV+':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'WCV+':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], Midpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))





exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()

In [36]:


import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
min_entries = {}
max_entries = {}

row = 2
for idx, label in enumerate(input_labels):
    ttk.Label(left_frame, text=label).grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)

    min_var = tk.StringVar(value=min_val)
    min_entry = ttk.Entry(left_frame, textvariable=min_var, width=10, state='readonly')
    min_entry.grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
    min_entries[label] = min_entry

    max_var = tk.StringVar(value=max_val)
    max_entry = ttk.Entry(left_frame, textvariable=max_var, width=10, state='readonly')
    max_entry.grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    max_entries[label] = max_entry

    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': 'jackknife',
    'J+': 'jackknife_plus',
    'CV': 'CV',
    'CV+': 'CV_plus',
    'WJ+': 'weighted_jackknife_plus',
    'WCV+': 'weighted_CV_plus'
}

selected_method = tk.StringVar()
selected_method.set('J')

for idx, (acronym, method) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=acronym, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Output", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], \nMidpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))

exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()


In [19]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
min_entries = {}
max_entries = {}

row = 2
for idx, label in enumerate(input_labels):
    ttk.Label(left_frame, text=label).grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)

    min_var = tk.StringVar(value=min_val)
    min_entry = ttk.Entry(left_frame, textvariable=min_var, width=10, state='normal')
    min_entry.grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
    min_entries[label] = min_entry

    max_var = tk.StringVar(value=max_val)
    max_entry = ttk.Entry(left_frame, textvariable=max_var, width=10, state='normal')
    max_entry.grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    max_entries[label] = max_entry

    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': 'jackknife',
    'J+': 'jackknife_plus',
    'CV': 'CV',
    'CV+': 'CV_plus',
    'WJ+': 'weighted_jackknife_plus',
    'WCV+': 'weighted_CV_plus'
}

selected_method = tk.StringVar()
selected_method.set('J')

for idx, (acronym, method) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=acronym, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Output", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], \nMidpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))

exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()


In [23]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
min_entries = {}
max_entries = {}

row = 2
for idx, label in enumerate(input_labels):
    ttk.Label(left_frame, text=label).grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)

    min_var = tk.StringVar(value=min_val)
    min_entry = ttk.Entry(left_frame, textvariable=min_var, width=10)
    min_entry.grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
    min_entries[label] = min_entry

    max_var = tk.StringVar(value=max_val)
    max_entry = ttk.Entry(left_frame, textvariable=max_var, width=10)
    max_entry.grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    max_entries[label] = max_entry

    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': 'jackknife',
    'J+': 'jackknife_plus',
    'CV': 'CV',
    'CV+': 'CV_plus',
    'WJ+': 'weighted_jackknife_plus',
    'WCV+': 'weighted_CV_plus'
}

selected_method = tk.StringVar()
selected_method.set('J')

for idx, (acronym, method) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=acronym, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Output", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], Midpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))

exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()


In [29]:
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Define model pipeline
scaler = StandardScaler()
pipeline = Pipeline([
    ("scaler", scaler),
    ("xgb", XGBRegressor(
        objective='reg:squarederror',
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Define widgets for input features
input_labels = [
    'C', 'FLA', 'W', 'CA', 'RA', 'RWA', 'FA', 'SP', 'CS', 'CC', 'ET', 'T', 'RH'
]

# Statistical summary values for default, min, and max values
stat_summary = {
    'C': (339.33, 133, 500),
    'FLA': (34.37, 0, 225.50),
    'W': (180.02, 46.56, 280),
    'CA': (528.23, 0, 1311),
    'RA': (538.35, 0, 1280),
    'RWA': (3.25, 0.34, 9.90),
    'FA': (659.05, 0, 998),
    'SP': (1.46, 0.40, 7.31),
    'CS': (41.69, 18.00, 72.60),
    'CC': (8.19, 0, 20),
    'ET': (147.91, 0, 3650),
    'T': (21.78, 0, 30),
    'RH': (63.51, 0, 78.30)
}

root = tk.Tk()
root.title("RAC Carbonation Depth Prediction Interval Model")

main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Left frame for input features
left_frame = ttk.Frame(main_frame, padding="10")
left_frame.grid(row=0, column=0, sticky=(tk.N, tk.S), padx=(0, 20))

# Input features title
input_features_title = ttk.Label(left_frame, text="Input Features", font=("Helvetica", 12))
input_features_title.grid(row=0, column=0, columnspan=4, pady=(0, 10))

# Adding headers for Value, Minimum, and Maximum columns
ttk.Label(left_frame, text="Value").grid(row=1, column=1, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Minimum").grid(row=1, column=2, sticky=tk.W, padx=5, pady=5)
ttk.Label(left_frame, text="Maximum").grid(row=1, column=3, sticky=tk.W, padx=5, pady=5)

entries = {}
entry_vars = {}
min_entries = {}
max_entries = {}

row = 2
for idx, label in enumerate(input_labels):
    ttk.Label(left_frame, text=label).grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
    default, min_val, max_val = stat_summary[label]
    slider_var = tk.DoubleVar(value=default)
    entry_vars[label] = slider_var
    entry = ttk.Entry(left_frame, textvariable=slider_var, width=10)
    entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)

    min_var = tk.StringVar(value=min_val)
    min_entry = ttk.Entry(left_frame, textvariable=min_var, width=10)
    min_entry.grid(row=row, column=2, sticky=tk.W, padx=5, pady=5)
    min_entries[label] = min_entry

    max_var = tk.StringVar(value=max_val)
    max_entry = ttk.Entry(left_frame, textvariable=max_var, width=10)
    max_entry.grid(row=row, column=3, sticky=tk.W, padx=5, pady=5)
    max_entries[label] = max_entry

    entries[label] = entry
    row += 1

# Right frame for the rest of the GUI
right_frame = ttk.Frame(main_frame, padding="10")
right_frame.grid(row=0, column=1, sticky=(tk.N, tk.S, tk.E, tk.W))

# Title for GUI
gui_title = ttk.Label(right_frame, text="RAC Carbonation Depth Prediction Interval Model", font=("Helvetica", 16))
gui_title.grid(row=0, column=0, columnspan=4, pady=(0, 20))

# Confidence level
confidence_label = ttk.Label(right_frame, text="Confidence Level")
confidence_label.grid(row=1, column=0, pady=(10, 5), sticky=tk.W)
confidence_var = tk.DoubleVar(value=0.95)
confidence_level = ttk.Scale(right_frame, from_=0.01, to=0.99, orient=tk.HORIZONTAL, variable=confidence_var)
confidence_level.grid(row=1, column=1, pady=(10, 5), sticky=(tk.W, tk.E))
confidence_entry = ttk.Entry(right_frame, textvariable=confidence_var, width=8)
confidence_entry.grid(row=1, column=2, pady=(10, 5), sticky=tk.W)

# Prediction Interval Method Buttons
methods_frame = ttk.LabelFrame(right_frame, text="Uncertainty Quantification Method", padding="10")
methods_frame.grid(row=2, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))

methods = {
    'J': 'jackknife',
    'J+': 'jackknife_plus',
    'CV': 'CV',
    'CV+': 'CV_plus',
    'WJ+': 'weighted_jackknife_plus',
    'WCV+': 'weighted_CV_plus'
}

selected_method = tk.StringVar()
selected_method.set('J')

for idx, (acronym, method) in enumerate(methods.items()):
    ttk.Radiobutton(methods_frame, text=acronym, variable=selected_method, value=method).grid(row=0, column=idx, padx=5)

# Output display
output_frame = ttk.LabelFrame(right_frame, text="Output", padding="10")
output_frame.grid(row=3, column=0, columnspan=4, pady=(10, 5), sticky=(tk.W, tk.E))
output_text = tk.Text(output_frame, height=3, width=50)
output_text.grid(row=0, column=0, sticky=(tk.W, tk.E))

# Function to compute prediction interval
def display_results():
    Xtest = np.array([[entry_vars[label].get() for label in input_labels]])
    alpha = 1 - confidence_var.get()
    X_train_data = X_train  # Placeholder for your training data
    y_train_data = y_train  # Placeholder for your training target

    method = selected_method.get()
    if method == 'jackknife':
        PIs = jackknife(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'jackknife_plus':
        PIs = jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_jackknife_plus':
        PIs = weighted_jackknife_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV':
        PIs = CV(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'CV_plus':
        PIs = CV_plus(X_train_data, y_train_data, Xtest, alpha)
    elif method == 'weighted_CV_plus':
        PIs = weighted_CV_plus(X_train_data, y_train_data, Xtest, alpha)

    formatted_PIs = [[f"{value:.2f}" for value in pi] for pi in PIs]
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"Prediction Interval = [{formatted_PIs[0][0]} , {formatted_PIs[0][1]}], Midpoint = {(PIs[0][0] + PIs[0][1]) / 2:.2f}")

# Buttons
compute_button = ttk.Button(right_frame, text="Compute", command=display_results)
compute_button.grid(row=4, column=0, pady=(20, 5))

reset_button = ttk.Button(right_frame, text="Reset", command=lambda: [entry_vars[label].set(stat_summary[label][0]) for label in input_labels])
reset_button.grid(row=4, column=1, pady=(20, 5))

exit_button = ttk.Button(right_frame, text="Exit", command=root.destroy)
exit_button.grid(row=4, column=2, pady=(20, 5))

root.mainloop()
